In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import multivariate_normal as mvn

from cr.mcr import *

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

# set plot parameters
params = {'legend.fontsize': 18,
          'figure.figsize': (16, 12),
          'lines.linewidth': 4,
          'axes.labelsize': 24,
          'axes.titlesize':24,
          'axes.linewidth':5,
          'xtick.labelsize':20,
          'ytick.labelsize':20}
plt.rcParams.update(params)
plt.style.use('seaborn-colorblind')
plt.rcParams['pdf.fonttype'] = 42

np.random.seed(123)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-j3yniibd because the default path (/home/jaron/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Set parameters of script

In [2]:
# define k for k-fold (n_splits = k)
n_splits = 20

# number of times to run k-fold x-validation 
# each trial uses different random partitioning of data
n_trials = 10

# import data

In [3]:
df = pd.read_csv("Data/2021_02_19_MultifunctionalDynamicData.csv")

all_experiments = df.Experiments.values
unique_experiments = np.unique(all_experiments)

df.head()

,Experiments,Time,PC_OD,PJ_OD,BV_OD,BF_OD,BO_OD,BT_OD,BC_OD,BY_OD,...,CG_OD,ER_OD,RI_OD,CC_OD,DL_OD,DF_OD,Butyrate,Acetate,Lactate,Succinate
0,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,0,0.000471,0.0,0.000471,0.000000,0.000000,0.000471,0.000471,0.0,...,0.000471,0.000471,0.000471,0.0,0.0,0.000471,0.000000,0.000000,28.000000,0.000000
1,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,16,0.465116,0.0,0.029207,0.000000,0.000000,0.249717,0.500651,0.0,...,0.024339,0.327601,0.001460,0.0,0.0,0.392830,23.092697,47.849302,18.910852,26.141885
2,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,32,0.104523,0.0,0.027928,0.000000,0.000000,0.220107,0.380210,0.0,...,0.020739,0.293384,0.000830,0.0,0.0,0.280111,23.996267,38.915218,17.977137,26.884748
3,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,48,0.124852,0.0,0.012194,0.000000,0.000000,0.268268,0.333970,0.0,...,0.023660,0.188188,0.002366,0.0,0.0,0.224952,24.839219,34.325914,19.406971,31.628061
4,BV-BF-BO-BT-BU-DP-BL-BA-BP-EL-FP-CH-AC-BH-CG-E...,0,0.000000,0.0,0.000388,0.000388,0.000388,0.000388,0.000000,0.0,...,0.000388,0.000388,0.000388,0.0,0.0,0.000000,0.000000,0.000000,28.000000,0.000000


### Define which variables are species and which are metabolites

In [4]:
# specify species and metabolite names 
species = df.columns.values[2:-4]
metabolites =  df.columns.values[-4:]

In [5]:
# perform k-fold x-validation several times (n_trials) with randomized partitions of data
for k in range(4, n_trials):
    
    # initialize dataframe to save predictions
    df_kfold = pd.DataFrame()

    # object to get indices of kfold splits
    kf = KFold(n_splits=n_splits, random_state=k, shuffle=True)

    # loop over k-fold splits
    for train_index, test_index in kf.split(unique_experiments):

        # update index values to pull trajectories 
        train_index = np.in1d(all_experiments, unique_experiments[train_index])
        test_index = np.in1d(all_experiments, unique_experiments[test_index])

        # pull train and test indices
        train_df  = df.iloc[train_index].copy() 
        test_df   = df.iloc[test_index].copy() 

        # scale metabolites based on training data values 
        train_df_scaled = train_df.copy()
        test_df_scaled  = test_df.copy()
        train_df_scaled[metabolites] /= train_df[metabolites].max()
        test_df_scaled[metabolites]  /= train_df[metabolites].max()
        
        # init model 
        model = CR(dataframe=train_df_scaled, species=species, resources=metabolites, r0=1.)

        # fit to data 
        model.fit(lr=1e-1, map_tol=1e-3, evd_tol=1e-3)

        # make predictions
        pred_df_kfold = model.predict_df(test_df_scaled, species, metabolites)
        
        # scale back resources
        for i, variable in enumerate(metabolites):
            pred_df_kfold[variable + " true"] *= train_df[variable].max()
            pred_df_kfold[variable + " pred"] *= train_df[variable].max()
            pred_df_kfold[variable + " stdv"] *= train_df[variable].max()
            
        # save kfold predictions
        df_kfold = pd.concat((df_kfold, pred_df_kfold))
        df_kfold.to_csv(f"Fig2/Kfold/MCR_{n_splits}_fold_{k+1}.csv", index=False)

Total samples: 182, Number of parameters: 300, Initial regularization: 1.00e+00
Epoch: 0, Loss: 380.03038, Residuals: -4.49660, Convergence:   inf
Epoch: 1, Loss: 354.27073, Residuals: -4.37526, Convergence: 0.072712
Epoch: 2, Loss: 333.20754, Residuals: -4.20953, Convergence: 0.063213
Epoch: 3, Loss: 317.17474, Residuals: -4.04488, Convergence: 0.050549
Epoch: 4, Loss: 304.94271, Residuals: -3.90037, Convergence: 0.040113
Epoch: 5, Loss: 295.24367, Residuals: -3.77320, Convergence: 0.032851
Epoch: 6, Loss: 287.37163, Residuals: -3.66282, Convergence: 0.027393
Epoch: 7, Loss: 280.83761, Residuals: -3.56855, Convergence: 0.023266
Epoch: 8, Loss: 275.27863, Residuals: -3.48814, Convergence: 0.020194
Epoch: 9, Loss: 270.43761, Residuals: -3.41916, Convergence: 0.017901
Epoch: 10, Loss: 266.13117, Residuals: -3.35948, Convergence: 0.016182
Epoch: 11, Loss: 262.22610, Residuals: -3.30730, Convergence: 0.014892
Epoch: 12, Loss: 258.62479, Residuals: -3.26111, Convergence: 0.013925
Epoch: 13,

Epoch: 111, Loss: 1043.99831, Residuals: -1.31693, Convergence: 0.005804
Epoch: 112, Loss: 1038.03641, Residuals: -1.31167, Convergence: 0.005743
Epoch: 113, Loss: 1032.07751, Residuals: -1.30568, Convergence: 0.005774
Epoch: 114, Loss: 1026.07593, Residuals: -1.29902, Convergence: 0.005849
Epoch: 115, Loss: 1020.04860, Residuals: -1.29176, Convergence: 0.005909
Epoch: 116, Loss: 1014.07087, Residuals: -1.28399, Convergence: 0.005895
Epoch: 117, Loss: 1008.25040, Residuals: -1.27583, Convergence: 0.005773
Epoch: 118, Loss: 1002.68634, Residuals: -1.26738, Convergence: 0.005549
Epoch: 119, Loss: 997.44151, Residuals: -1.25875, Convergence: 0.005258
Epoch: 120, Loss: 992.53734, Residuals: -1.25004, Convergence: 0.004941
Epoch: 121, Loss: 987.96639, Residuals: -1.24132, Convergence: 0.004627
Epoch: 122, Loss: 983.70526, Residuals: -1.23266, Convergence: 0.004332
Epoch: 123, Loss: 979.72579, Residuals: -1.22410, Convergence: 0.004062
Epoch: 124, Loss: 975.99997, Residuals: -1.21570, Conver

Epoch: 215, Loss: 2455.79429, Residuals: -0.93282, Convergence: 0.002402
Epoch: 216, Loss: 2450.77819, Residuals: -0.93027, Convergence: 0.002047
Epoch: 217, Loss: 2446.46767, Residuals: -0.92808, Convergence: 0.001762
Epoch: 218, Loss: 2442.72463, Residuals: -0.92621, Convergence: 0.001532
Epoch: 219, Loss: 2439.43965, Residuals: -0.92463, Convergence: 0.001347
Epoch: 220, Loss: 2436.52854, Residuals: -0.92330, Convergence: 0.001195
Epoch: 221, Loss: 2433.92323, Residuals: -0.92219, Convergence: 0.001070
Epoch: 222, Loss: 2431.57157, Residuals: -0.92127, Convergence: 0.000967
Evidence 14807.212

Epoch: 222, Evidence: 14807.21191, Convergence: 0.006462
Updating hyper-parameters...
Total samples: 182, Updated regularization: 2.64e-01
Epoch: 222, Loss: 2470.24032, Residuals: -0.92127, Convergence:   inf
Epoch: 223, Loss: 2466.14171, Residuals: -0.91883, Convergence: 0.001662
Epoch: 224, Loss: 2462.71069, Residuals: -0.91681, Convergence: 0.001393
Epoch: 225, Loss: 2459.77662, Residuals: 

Epoch: 57, Loss: 161.23138, Residuals: -1.81051, Convergence: 0.006681
Epoch: 58, Loss: 160.20800, Residuals: -1.78530, Convergence: 0.006388
Epoch: 59, Loss: 159.23623, Residuals: -1.76080, Convergence: 0.006103
Epoch: 60, Loss: 158.31371, Residuals: -1.73701, Convergence: 0.005827
Epoch: 61, Loss: 157.43790, Residuals: -1.71394, Convergence: 0.005563
Epoch: 62, Loss: 156.60617, Residuals: -1.69158, Convergence: 0.005311
Epoch: 63, Loss: 155.81587, Residuals: -1.66992, Convergence: 0.005072
Epoch: 64, Loss: 155.06435, Residuals: -1.64894, Convergence: 0.004846
Epoch: 65, Loss: 154.34901, Residuals: -1.62864, Convergence: 0.004635
Epoch: 66, Loss: 153.66730, Residuals: -1.60898, Convergence: 0.004436
Epoch: 67, Loss: 153.01683, Residuals: -1.58996, Convergence: 0.004251
Epoch: 68, Loss: 152.39544, Residuals: -1.57154, Convergence: 0.004078
Epoch: 69, Loss: 151.80115, Residuals: -1.55372, Convergence: 0.003915
Epoch: 70, Loss: 151.23227, Residuals: -1.53646, Convergence: 0.003762
Epoch:

Epoch: 165, Loss: 2260.57717, Residuals: -1.08755, Convergence: 0.010263
Epoch: 166, Loss: 2240.92416, Residuals: -1.08549, Convergence: 0.008770
Epoch: 167, Loss: 2224.11838, Residuals: -1.08314, Convergence: 0.007556
Epoch: 168, Loss: 2209.62185, Residuals: -1.08055, Convergence: 0.006561
Epoch: 169, Loss: 2197.00356, Residuals: -1.07775, Convergence: 0.005743
Epoch: 170, Loss: 2185.91591, Residuals: -1.07479, Convergence: 0.005072
Epoch: 171, Loss: 2176.07480, Residuals: -1.07168, Convergence: 0.004522
Epoch: 172, Loss: 2167.25118, Residuals: -1.06845, Convergence: 0.004071
Epoch: 173, Loss: 2159.25668, Residuals: -1.06509, Convergence: 0.003702
Epoch: 174, Loss: 2151.94152, Residuals: -1.06164, Convergence: 0.003399
Epoch: 175, Loss: 2145.19043, Residuals: -1.05808, Convergence: 0.003147
Epoch: 176, Loss: 2138.92544, Residuals: -1.05445, Convergence: 0.002929
Epoch: 177, Loss: 2133.09558, Residuals: -1.05076, Convergence: 0.002733
Epoch: 178, Loss: 2127.67102, Residuals: -1.04707, 

...

Epoch: 67, Loss: 153.66919, Residuals: -1.58699, Convergence: 0.004469
Epoch: 68, Loss: 153.01102, Residuals: -1.56801, Convergence: 0.004302
Epoch: 69, Loss: 152.38013, Residuals: -1.54962, Convergence: 0.004140
Epoch: 70, Loss: 151.77547, Residuals: -1.53180, Convergence: 0.003984
Epoch: 71, Loss: 151.19611, Residuals: -1.51454, Convergence: 0.003832
Epoch: 72, Loss: 150.64124, Residuals: -1.49785, Convergence: 0.003683
Epoch: 73, Loss: 150.11010, Residuals: -1.48171, Convergence: 0.003538
Epoch: 74, Loss: 149.60198, Residuals: -1.46612, Convergence: 0.003396
Epoch: 75, Loss: 149.11617, Residuals: -1.45107, Convergence: 0.003258
Epoch: 76, Loss: 148.65197, Residuals: -1.43655, Convergence: 0.003123
Epoch: 77, Loss: 148.20867, Residuals: -1.42256, Convergence: 0.002991
Epoch: 78, Loss: 147.78553, Residuals: -1.40909, Convergence: 0.002863
Epoch: 79, Loss: 147.38184, Residuals: -1.39612, Convergence: 0.002739
Epoch: 80, Loss: 146.99686, Residuals: -1.38365, Convergence: 0.002619
Epoch:

Epoch: 175, Loss: 2232.34660, Residuals: -1.05879, Convergence: 0.006200
Epoch: 176, Loss: 2220.39204, Residuals: -1.05621, Convergence: 0.005384
Epoch: 177, Loss: 2209.94284, Residuals: -1.05358, Convergence: 0.004728
Epoch: 178, Loss: 2200.68746, Residuals: -1.05089, Convergence: 0.004206
Epoch: 179, Loss: 2192.37107, Residuals: -1.04812, Convergence: 0.003793
Epoch: 180, Loss: 2184.79986, Residuals: -1.04525, Convergence: 0.003465
Epoch: 181, Loss: 2177.83351, Residuals: -1.04227, Convergence: 0.003199
Epoch: 182, Loss: 2171.38390, Residuals: -1.03921, Convergence: 0.002970
Epoch: 183, Loss: 2165.39518, Residuals: -1.03609, Convergence: 0.002766
Epoch: 184, Loss: 2159.83081, Residuals: -1.03296, Convergence: 0.002576
Epoch: 185, Loss: 2154.66034, Residuals: -1.02985, Convergence: 0.002400
Epoch: 186, Loss: 2149.85438, Residuals: -1.02680, Convergence: 0.002235
Epoch: 187, Loss: 2145.38556, Residuals: -1.02384, Convergence: 0.002083
Epoch: 188, Loss: 2141.22633, Residuals: -1.02097, 

Epoch: 11, Loss: 265.22675, Residuals: -3.34887, Convergence: 0.014896
Epoch: 12, Loss: 261.59617, Residuals: -3.30174, Convergence: 0.013879
Epoch: 13, Loss: 258.21470, Residuals: -3.25950, Convergence: 0.013096
Epoch: 14, Loss: 255.03100, Residuals: -3.22100, Convergence: 0.012484
Epoch: 15, Loss: 252.01058, Residuals: -3.18528, Convergence: 0.011985
Epoch: 16, Loss: 249.13405, Residuals: -3.15170, Convergence: 0.011546
Epoch: 17, Loss: 246.38878, Residuals: -3.11986, Convergence: 0.011142
Epoch: 18, Loss: 243.75528, Residuals: -3.08938, Convergence: 0.010804
Epoch: 19, Loss: 241.20265, Residuals: -3.05978, Convergence: 0.010583
Epoch: 20, Loss: 238.69412, Residuals: -3.03048, Convergence: 0.010509
Epoch: 21, Loss: 236.19428, Residuals: -3.00090, Convergence: 0.010584
Epoch: 22, Loss: 233.67295, Residuals: -2.97059, Convergence: 0.010790
Epoch: 23, Loss: 231.10019, Residuals: -2.93917, Convergence: 0.011133
Epoch: 24, Loss: 228.43163, Residuals: -2.90613, Convergence: 0.011682
Epoch:

Epoch: 123, Loss: 1007.07085, Residuals: -1.26467, Convergence: 0.004457
Epoch: 124, Loss: 1002.89090, Residuals: -1.25624, Convergence: 0.004168
Epoch: 125, Loss: 998.99710, Residuals: -1.24790, Convergence: 0.003898
Epoch: 126, Loss: 995.36212, Residuals: -1.23968, Convergence: 0.003652
Epoch: 127, Loss: 991.95935, Residuals: -1.23163, Convergence: 0.003430
Epoch: 128, Loss: 988.76491, Residuals: -1.22377, Convergence: 0.003231
Epoch: 129, Loss: 985.75814, Residuals: -1.21612, Convergence: 0.003050
Epoch: 130, Loss: 982.92245, Residuals: -1.20871, Convergence: 0.002885
Epoch: 131, Loss: 980.24372, Residuals: -1.20154, Convergence: 0.002733
Epoch: 132, Loss: 977.71066, Residuals: -1.19463, Convergence: 0.002591
Epoch: 133, Loss: 975.31390, Residuals: -1.18798, Convergence: 0.002457
Epoch: 134, Loss: 973.04503, Residuals: -1.18161, Convergence: 0.002332
Epoch: 135, Loss: 970.89639, Residuals: -1.17552, Convergence: 0.002213
Epoch: 136, Loss: 968.86083, Residuals: -1.16970, Convergence:

Evidence 14889.920

Epoch: 219, Evidence: 14889.91992, Convergence: 0.000857
Updating hyper-parameters...
Total samples: 183, Updated regularization: 1.78e-01
Epoch: 219, Loss: 2497.46910, Residuals: -0.94473, Convergence:   inf
Epoch: 220, Loss: 2492.90647, Residuals: -0.94158, Convergence: 0.001830
Epoch: 221, Loss: 2489.31904, Residuals: -0.93956, Convergence: 0.001441
Epoch: 222, Loss: 2486.35146, Residuals: -0.93804, Convergence: 0.001194
Epoch: 223, Loss: 2483.79393, Residuals: -0.93703, Convergence: 0.001030
Epoch: 224, Loss: 2481.52428, Residuals: -0.93635, Convergence: 0.000915
Evidence 14910.696

Epoch: 224, Evidence: 14910.69629, Convergence: 0.002250
Updating hyper-parameters...
Total samples: 183, Updated regularization: 1.47e-01
Epoch: 224, Loss: 2497.53050, Residuals: -0.93635, Convergence:   inf
Epoch: 225, Loss: 2494.62349, Residuals: -0.93365, Convergence: 0.001165
Epoch: 226, Loss: 2492.24858, Residuals: -0.93219, Convergence: 0.000953
Evidence 14922.049

Epoch: 226,